# RAG - Indexación con Weaviate

**Migración desde Qdrant a Weaviate:**
- ✅ Persistencia automática más confiable
- ✅ Mejor manejo de concurrencia
- ✅ API más estable
- ✅ Sin problemas de corrupción de vectores

In [1]:
import os
import logging
import warnings
import torch
import gc
from pathlib import Path
from dotenv import load_dotenv

# LangChain imports
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_weaviate import WeaviateVectorStore
from langchain.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings.base import Embeddings
from langchain_core.documents import Document

# Weaviate imports
import weaviate
from weaviate.classes.config import Property, DataType


from sentence_transformers import SentenceTransformer
from langsmith import Client
from langsmith.evaluation import evaluate as ls_evaluate, LangChainStringEvaluator

# Configuración
warnings.filterwarnings('ignore', category=FutureWarning)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
load_dotenv(override=True)

# Detectar dispositivo
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Usando dispositivo: {device}')

# Para avisos de Triton
logging.getLogger("xformers").setLevel(logging.ERROR)

# Limpiar memoria GPU si está disponible
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    print(f'Memoria GPU limpiada')

e:\ANACONDA\envs\rag\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Usando dispositivo: cuda
Memoria GPU limpiada


In [6]:
# Configuración
WEAVIATE_URL = os.getenv('WEAVIATE_URL', 'http://localhost:8080')

# Directorios de documentos
PDF_FOLDER = Path("pdf_articulos_idearq-v2/ingesta")
TABLE_PDF = Path("pdf_articulos_idearq-v2/1186_Dataciones-v2.pdf")

# CHUNKING OPTIMIZADO
CHUNK_CONFIG = {
    "chunk_size": 800,
    "chunk_overlap": 50,
    "length_function": len,
    "add_start_index": True,
    "separators": ["\n\n\n", "\n\n", "\n", ". ", ", ", " ", ""],
}

# LIMITACIÓN PARA 525 ARTÍCULOS
MAX_CHUNKS_PER_DOC = 15
MIN_CHUNK_LENGTH = 100
MAX_CHUNK_LENGTH = 2000
ENABLE_ADAPTIVE_LIMIT = True

print(f'Configuración cargada:')
print(f'   - Weaviate URL: {WEAVIATE_URL}')
print(f'   - PDF Folder: {PDF_FOLDER.resolve()}')
print(f'   - Table PDF: {TABLE_PDF.resolve()}')
print(f'   - Chunk size: {CHUNK_CONFIG["chunk_size"]}, overlap: {CHUNK_CONFIG["chunk_overlap"]} ({CHUNK_CONFIG["chunk_overlap"]/CHUNK_CONFIG["chunk_size"]*100:.1f}%)')
print(f'   - Max chunks/doc: {MAX_CHUNKS_PER_DOC} (adaptativo: {ENABLE_ADAPTIVE_LIMIT})')
print(f'   - Estimación para 525 artículos: ~{525 * MAX_CHUNKS_PER_DOC * 0.7:,.0f} chunks')

Configuración cargada:
   - Weaviate URL: http://localhost:8080
   - PDF Folder: C:\Users\elevi\Documents\CSIC\RAG\pdf_articulos_idearq-v2\ingesta
   - Table PDF: C:\Users\elevi\Documents\CSIC\RAG\pdf_articulos_idearq-v2\1186_Dataciones-v2.pdf
   - Chunk size: 800, overlap: 50 (6.2%)
   - Max chunks/doc: 15 (adaptativo: True)
   - Estimación para 525 artículos: ~5,512 chunks


In [7]:
# Implementación personalizada para E5 Instruct
from typing import List, Dict, Any, Optional

class E5InstructEmbeddings(Embeddings):
    def __init__(self, model_name="intfloat/multilingual-e5-large-instruct", device=None):
        """Inicializa el modelo E5 Instruct con prefijos específicos."""
        if device is None:
            device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        self.device = device
        print(f'Cargando modelo E5 en {device}...')
        self.model = SentenceTransformer(model_name, device=device)
        print(f'Modelo E5 cargado correctamente')
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Para documentos - añadir prefix 'passage:'"""
        prefixed_texts = [f"passage: {text}" for text in texts]
        embeddings = self.model.encode(prefixed_texts, device=self.device, show_progress_bar=False)
        return embeddings.tolist()
    
    def embed_query(self, text: str) -> List[float]:
        """Para queries - añadir prefix 'query:'"""
        prefixed_text = f"query: {text}"
        embedding = self.model.encode([prefixed_text], device=self.device, show_progress_bar=False)[0]
        return embedding.tolist()

print('Clase E5InstructEmbeddings definida')

Clase E5InstructEmbeddings definida


In [8]:
# Configuración de modelos de embedding MEJORADA
print('Inicializando modelos de embedding...')

model_kwargs_gte = {'device': device, 'trust_remote_code': True}
model_kwargs_mini = {'device': device}

EMBEDDING_MODELS = {
    "all-MiniLM-L6-v2": {
        "model": HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs=model_kwargs_mini
        ),
        "vector_size": 384,
        "class_name": "IdearqAllMiniLM",
        "max_tokens": 512,
        "batch_size": 10
    },
    "gte-multilingual-base": {
        "model": HuggingFaceEmbeddings(
            model_name="Alibaba-NLP/gte-multilingual-base",
            model_kwargs=model_kwargs_gte
        ),
        "vector_size": 768,
        "class_name": "IdearqGTE",
        "max_tokens": 512,
        "batch_size": 8
    },
    "e5-large-instruct": {
        "model": E5InstructEmbeddings(
            model_name="intfloat/multilingual-e5-large-instruct",
            device=device
        ),
        "vector_size": 1024,
        "class_name": "IdearqE5",
        "max_tokens": 1024,
        "batch_size": 5
    }
}

print('Modelos de embedding configurados:')
for name, config in EMBEDDING_MODELS.items():
    print(f'   - {name}: {config["vector_size"]}D, max_tokens: {config["max_tokens"]} → {config["class_name"]}')

2025-08-31 00:44:55,823 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Inicializando modelos de embedding...


2025-08-31 00:44:58,095 - INFO - Load pretrained SentenceTransformer: Alibaba-NLP/gte-multilingual-base
<frozen importlib._bootstrap>:530: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.15; use exec_module() instead
Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2025-08-31 00:45:07,690 - INFO - Load pretrained SentenceTransformer: intfloat/multilingual-e5-large-instruct


Cargando modelo E5 en cuda...
Modelo E5 cargado correctamente
Modelos de embedding configurados:
   - all-MiniLM-L6-v2: 384D, max_tokens: 512 → IdearqAllMiniLM
   - gte-multilingual-base: 768D, max_tokens: 512 → IdearqGTE
   - e5-large-instruct: 1024D, max_tokens: 1024 → IdearqE5


In [9]:
# FUNCIONES DE CARGA MEJORADAS (IGUAL QUE QDRANT)
def load_documents_pymupdf_improved(folder_path: Path) -> List[Document]:
    """Cargar PDFs con PyMuPDFLoader - preservando metadatos del archivo."""
    import re

    all_docs = []

    if not folder_path.is_dir():
        logging.error(f"El directorio no existe: {folder_path.resolve()}")
        return all_docs

    pdf_files = list(folder_path.glob("*.pdf"))
    logging.info(f"Iniciando carga MEJORADA desde: {folder_path.resolve()}")
    logging.info(f"Encontrados {len(pdf_files)} archivos PDF")

    for i, pdf_file in enumerate(pdf_files, 1):
        try:
            loader = PyMuPDFLoader(str(pdf_file))
            docs_from_file = loader.load()

            if not docs_from_file:
                logging.warning(f"Archivo vacío: {pdf_file.name}")
                continue

            # COMBINAR páginas manteniendo metadatos
            combined_content = "\n\n".join([doc.page_content for doc in docs_from_file])
            
            first_doc_metadata = docs_from_file[0].metadata
            doc_title = first_doc_metadata.get('title', pdf_file.stem)
            
            combined_doc = Document(
                page_content=combined_content,
                metadata={
                    'source': str(pdf_file),
                    'filename': pdf_file.name,
                    'title': doc_title,
                    'doc_title': doc_title,
                    'total_pages': len(docs_from_file),
                    'file_type': 'pdf',
                    'loader': 'PyMuPDFLoader_improved'
                }
            )

            # LIMPIAR caracteres de control
            combined_doc.page_content = re.sub(r'[\x00-\x1f\x7f-\x9f]', ' ', combined_doc.page_content)
            combined_doc.page_content = re.sub(r'\s+', ' ', combined_doc.page_content.strip())

            all_docs.append(combined_doc)

            if i % 10 == 0:
                logging.info(f"Procesados {i}/{len(pdf_files)} archivos...")

        except Exception as e:
            logging.warning(f"Error cargando '{pdf_file.name}': {e}")

    logging.info(f"Carga completada: {len(all_docs)} documentos")
    return all_docs


def load_table_pdfplumber_optimized(pdf_path: Path) -> List[Document]:
    """Cargar tabla PDF con PDFPlumberLoader."""
    import re

    if not pdf_path.exists():
        logging.error(f"El archivo no existe: {pdf_path.resolve()}")
        return []

    logging.info(f"Cargando tabla desde: {pdf_path.resolve()}")
    try:
        loader = PDFPlumberLoader(str(pdf_path))
        docs = loader.load()

        for doc in docs:
            doc.page_content = re.sub(r'[\x00-\x1f\x7f-\x9f]', ' ', doc.page_content)
            doc.page_content = re.sub(r'\s+', ' ', doc.page_content.strip())
            
            doc.metadata.update({
                'filename': pdf_path.name,
                'source': str(pdf_path),
                'file_type': 'table_pdf',
                'loader': 'PDFPlumberLoader_optimized'
            })

        logging.info(f"Tabla cargada: {len(docs)} páginas")
        return docs
    except Exception as e:
        logging.error(f"Error cargando tabla: {e}")
        return []

print('Funciones de carga definidas')

Funciones de carga definidas


In [10]:
# FUNCIÓN DE CHUNKING (IGUAL QUE ANTES)
def split_documents_with_smart_limiting(docs: List[Document], 
                                       text_splitter: RecursiveCharacterTextSplitter,
                                       max_chunks_per_doc: int = MAX_CHUNKS_PER_DOC,
                                       enable_adaptive: bool = ENABLE_ADAPTIVE_LIMIT) -> List[Document]:
    """Chunking con limitación inteligente y adaptativa."""
    
    def get_adaptive_chunk_limit(doc_length: int, base_limit: int) -> int:
        if not enable_adaptive:
            return base_limit
        
        if doc_length < 3000:
            return min(base_limit, 5)
        elif doc_length < 8000:
            return min(base_limit, 8)
        elif doc_length < 20000:
            return min(base_limit, 12)
        elif doc_length < 50000:
            return base_limit
        else:
            return int(base_limit * 1.3)
    
    def score_chunk_quality(chunk: Document, doc_title: str, chunk_idx: int, total_chunks: int) -> float:
        content = chunk.page_content.lower()
        score = 0.0
        
        if MIN_CHUNK_LENGTH <= len(content) <= CHUNK_CONFIG["chunk_size"] * 1.2:
            score += 1.0
        
        academic_keywords = [
            'abstract', 'introduction', 'results', 'conclusion', 'methodology', 'discussion', 
            'analysis', 'research', 'study', 'arqueología', 'neolítico', 'datación', 
            'isótopos', 'radiocarbono', 'cronología', 'yacimiento', 'excavación'
        ]
        score += sum(0.15 for keyword in academic_keywords if keyword in content)
        
        if chunk_idx < 2 or chunk_idx >= total_chunks - 2:
            score += 0.8
        elif chunk_idx < total_chunks * 0.2:
            score += 0.4
        elif chunk_idx > total_chunks * 0.8:
            score += 0.4
        
        import re
        if re.search(r'\d{3,4}', content):
            score += 0.3
        if re.search(r'cal\s*(BP|BC|AD)', content, re.IGNORECASE):
            score += 0.5
        
        if len(content) < MIN_CHUNK_LENGTH:
            score -= 2.0
        elif len(content) > MAX_CHUNK_LENGTH:
            score -= 1.0
            
        return score
    
    all_chunks = []
    total_chunks_reduced = 0
    
    print(f"Aplicando chunking a {len(docs)} documentos...")
    
    for doc_idx, doc in enumerate(docs):
        doc_length = len(doc.page_content)
        raw_chunks = text_splitter.split_documents([doc])
        adaptive_limit = get_adaptive_chunk_limit(doc_length, max_chunks_per_doc)
        
        if len(raw_chunks) <= adaptive_limit:
            selected_chunks = raw_chunks
        else:
            scored_chunks = []
            for i, chunk in enumerate(raw_chunks):
                quality_score = score_chunk_quality(chunk, doc.metadata.get('title', ''), i, len(raw_chunks))
                scored_chunks.append((chunk, quality_score))
            
            scored_chunks.sort(key=lambda x: x[1], reverse=True)
            selected_chunks = [chunk for chunk, score in scored_chunks[:adaptive_limit]]
            total_chunks_reduced += len(raw_chunks) - len(selected_chunks)
        
        for i, chunk in enumerate(selected_chunks):
            chunk.metadata.update({
                'chunk_index': i,
                'total_chunks_from_doc': len(selected_chunks),
                'doc_index': doc_idx
            })
        
        all_chunks.extend(selected_chunks)
    
    total_original = len(docs) * max_chunks_per_doc  # Estimado
    total_selected = len(all_chunks)
    
    print(f"\nCHUNKING COMPLETADO:")
    print(f"   • Documentos procesados: {len(docs)}")
    print(f"   • Chunks generados: {total_selected:,}")
    print(f"   • Promedio chunks/documento: {total_selected/len(docs):.1f}")
    
    return all_chunks

print('Función de chunking definida')

Función de chunking definida


In [11]:
print('CARGANDO DOCUMENTOS...')

regular_docs = load_documents_pymupdf_improved(PDF_FOLDER)

print('\nCargando documentos de tabla...')
table_docs = load_table_pdfplumber_optimized(TABLE_PDF)

# Combinar todos los documentos
all_documents = regular_docs + table_docs

print('\nRESUMEN DE CARGA:')
print(f'   • Documentos regulares: {len(regular_docs)}')
print(f'   • Documentos tabla: {len(table_docs)}')
print(f'   • TOTAL documentos: {len(all_documents)}')

# Verificar metadatos
if all_documents:
    sample_doc = all_documents[0]
    print(f'\nVERIFICACIÓN DE METADATOS:')
    print(f'   • Filename: {sample_doc.metadata.get("filename", "MISSING")}')
    print(f'   • Title: {sample_doc.metadata.get("title", "MISSING")}')

# Limpiar memoria
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()

2025-08-31 00:45:31,935 - INFO - Iniciando carga MEJORADA desde: C:\Users\elevi\Documents\CSIC\RAG\pdf_articulos_idearq-v2\ingesta
2025-08-31 00:45:31,938 - INFO - Encontrados 525 archivos PDF


CARGANDO DOCUMENTOS...


<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type swigvarlink has no __module__ attribute
2025-08-31 00:45:35,414 - INFO - Procesados 10/525 archivos...
2025-08-31 00:45:36,791 - INFO - Procesados 20/525 archivos...
2025-08-31 00:45:37,969 - INFO - Procesados 30/525 archivos...
2025-08-31 00:45:39,641 - INFO - Procesados 40/525 archivos...
2025-08-31 00:45:42,640 - INFO - Procesados 50/525 archivos...
2025-08-31 00:45:43,722 - INFO - Procesados 60/525 archivos...
2025-08-31 00:45:48,311 - INFO - Procesados 70/525 archivos...
2025-08-31 00:45:51,620 - INFO - Procesados 80/525 archivos...
2025-08-31 00:45:53,413 - INFO - Procesados 90/525 archivos...
2025-08-31 00:46:01,394 - INFO - Procesados 100/525 archivos...
2025-08-31 00:46:03,174 - INFO - Procesados


Cargando documentos de tabla...


2025-08-31 00:48:44,944 - INFO - Tabla cargada: 119 páginas



RESUMEN DE CARGA:
   • Documentos regulares: 525
   • Documentos tabla: 119
   • TOTAL documentos: 644

VERIFICACIÓN DE METADATOS:
   • Filename: 1000_oms_2017.pdf
   • Title: First Evidence of Collective Human Inhumation from the Cardial Neolithic (Cova Bonica, Barcelona, NE Iberian Peninsula)


In [12]:
# APLICAR CHUNKING
from collections import defaultdict, Counter
from langchain_text_splitters import RecursiveCharacterTextSplitter

print('\nAPLICANDO CHUNKING...')

text_splitter = RecursiveCharacterTextSplitter(**CHUNK_CONFIG)

all_chunks = split_documents_with_smart_limiting(
    docs=all_documents,
    text_splitter=text_splitter,
    max_chunks_per_doc=MAX_CHUNKS_PER_DOC
)

print(f'\nCHUNKS GENERADOS: {len(all_chunks)}')

# Limpiar memoria
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()


APLICANDO CHUNKING...
Aplicando chunking a 644 documentos...

CHUNKING COMPLETADO:
   • Documentos procesados: 644
   • Chunks generados: 9,491
   • Promedio chunks/documento: 14.7

CHUNKS GENERADOS: 9491


In [13]:
# CONECTAR A WEAVIATE Y CREAR CLASES
print('CONECTANDO A WEAVIATE...')

client = weaviate.connect_to_local(
    host="localhost",
    port=8080,
    grpc_port=50051
)

def create_weaviate_class(client, class_name: str, vector_size: int) -> bool:
    """Crear una clase en Weaviate para embeddings."""
    try:
        # Borrar clase si existe
        if client.collections.exists(class_name):
            print(f"Borrando clase existente: {class_name}")
            client.collections.delete(class_name)
        
        # Crear clase nueva
        print(f"Creando clase: {class_name} ({vector_size}D)")
        
        client.collections.create(
            name=class_name,
            properties=[
                Property(name="content", data_type=DataType.TEXT),
                Property(name="filename", data_type=DataType.TEXT),
                Property(name="title", data_type=DataType.TEXT),
                Property(name="source", data_type=DataType.TEXT),
                Property(name="chunk_index", data_type=DataType.INT),
                Property(name="doc_index", data_type=DataType.INT)
            ],
            # Configuración de vectores
            vector_index_config=weaviate.classes.config.Configure.VectorIndex.hnsw(),
            vectorizer_config=None  # Usaremos vectores externos
        )
        
        return True
        
    except Exception as e:
        print(f"Error creando clase {class_name}: {e}")
        return False

classes_created = []

for embedding_name, config in EMBEDDING_MODELS.items():
    class_name = config["class_name"]
    vector_size = config["vector_size"]
    
    if create_weaviate_class(client, class_name, vector_size):
        classes_created.append((embedding_name, class_name))
        print(f'✅ {class_name} creada')
    else:
        print(f'❌ Error creando {class_name}')

print(f'\nSe han creado {len(classes_created)} clases correctamente')

CONECTANDO A WEAVIATE...


2025-08-31 00:48:55,847 - INFO - HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
2025-08-31 00:48:55,928 - INFO - HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"
2025-08-31 00:48:56,187 - INFO - HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
2025-08-31 00:48:56,212 - INFO - HTTP Request: GET http://localhost:8080/v1/schema/IdearqAllMiniLM "HTTP/1.1 404 Not Found"
e:\ANACONDA\envs\rag\Lib\site-packages\weaviate\warnings.py:206: DeprecationWarning: Dep025: You are using the `vector_index_config` argument in `collection.config.create()`, which is deprecated.
            Use the `vector_config` argument instead defining `vector_index_config` as a sub-argument.
            
  warnings.warn(


Creando clase: IdearqAllMiniLM (384D)


2025-08-31 00:48:56,550 - INFO - HTTP Request: POST http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
2025-08-31 00:48:56,555 - INFO - HTTP Request: GET http://localhost:8080/v1/schema/IdearqGTE "HTTP/1.1 404 Not Found"


✅ IdearqAllMiniLM creada
Creando clase: IdearqGTE (768D)


2025-08-31 00:48:56,795 - INFO - HTTP Request: POST http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
2025-08-31 00:48:56,802 - INFO - HTTP Request: GET http://localhost:8080/v1/schema/IdearqE5 "HTTP/1.1 404 Not Found"


✅ IdearqGTE creada
Creando clase: IdearqE5 (1024D)


2025-08-31 00:48:57,044 - INFO - HTTP Request: POST http://localhost:8080/v1/schema "HTTP/1.1 200 OK"


✅ IdearqE5 creada

Se han creado 3 clases correctamente


In [14]:
# FUNCIÓN DE INDEXACIÓN PARA WEAVIATE
import numpy as np
from collections import Counter

def index_documents_weaviate(embedding_name: str, config: dict, chunks: List[Document]) -> bool:
    """Indexar documentos en Weaviate con validación."""
    import re
    
    def estimate_tokens(text: str) -> int:
        return len(text) // 3
    
    def validate_chunk_for_model(chunk: Document, max_tokens: int) -> tuple:
        content = chunk.page_content
        issues = []
        
        estimated_tokens = estimate_tokens(content)
        if estimated_tokens > max_tokens:
            issues.append(f"too_long_{estimated_tokens}_tokens")
        
        if len(content.strip()) < MIN_CHUNK_LENGTH:
            issues.append("too_short")
        
        if re.search(r'[\x00-\x1f\x7f-\x9f]', content):
            issues.append("control_chars")
        
        clean_content = re.sub(r'\s+', ' ', content.strip())
        if len(clean_content) < 20:
            issues.append("mostly_empty")
        
        return len(issues) == 0, issues
    
    class_name = config["class_name"]
    embedding_model = config["model"]
    max_tokens = config["max_tokens"]
    batch_size = config["batch_size"]
    
    print(f'\nINDEXANDO: {embedding_name}')
    print(f'   • Clase: {class_name}')
    print(f'   • Límite de tokens: {max_tokens}')
    print(f'   • Batch size: {batch_size}')
    print(f'   • Chunks a procesar: {len(chunks)}')
    
    # Filtrar chunks válidos
    valid_chunks = []
    skipped_stats = Counter()
    
    print('Validando chunks...')
    for chunk in chunks:
        is_valid, issues = validate_chunk_for_model(chunk, max_tokens)
        
        if is_valid:
            valid_chunks.append(chunk)
        else:
            for issue in issues:
                skipped_stats[issue] += 1
    
    print(f'\nVALIDACIÓN COMPLETADA:')
    print(f'   • Chunks válidos: {len(valid_chunks)}')
    print(f'   • Chunks saltados: {len(chunks) - len(valid_chunks)}')
    print(f'   • Tasa de éxito: {len(valid_chunks)/len(chunks)*100:.1f}%')
    
    if not valid_chunks:
        print('No hay chunks válidos para indexar')
        return False
    
    try:
        # Test de embedding
        print('Probando generación de embeddings...')
        test_embedding = embedding_model.embed_query("test arqueología neolítico")
        embedding_norm = np.linalg.norm(test_embedding)
        print(f'   • Test embedding: {len(test_embedding)}D, norma: {embedding_norm:.3f}')
        
        if embedding_norm < 0.1:
            print('Test embedding tiene norma muy baja, abortando')
            return False
        
        # Crear vector store
        vector_store = WeaviateVectorStore(
            client=client,
            index_name=class_name,
            text_key="content",
            embedding=embedding_model,
            attributes=["filename", "title", "source", "chunk_index", "doc_index"]
        )
        
        # Indexar chunks en batches
        print(f'Iniciando indexación de {len(valid_chunks)} chunks...')
        
        for i in range(0, len(valid_chunks), batch_size):
            batch = valid_chunks[i:i + batch_size]
            vector_store.add_documents(batch)
            
            if (i // batch_size + 1) % 10 == 0:
                print(f'   Procesados {i + len(batch)} chunks...')
        
        print(f'INDEXACIÓN EXITOSA: {embedding_name}')
        return True
        
    except Exception as e:
        print(f'Error durante indexación de {embedding_name}: {e}')
        return False
    
    finally:
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

print('Función de indexación Weaviate definida')

Función de indexación Weaviate definida


In [15]:
# INDEXAR TODOS LOS MODELOS EN WEAVIATE
print('\nINICIANDO INDEXACIÓN EN WEAVIATE')
print('=' * 70)

successful_indexes = []
failed_indexes = []

for embedding_name, config in EMBEDDING_MODELS.items():
    print(f'\nProcesando: {embedding_name}')
    
    success = index_documents_weaviate(
        embedding_name=embedding_name,
        config=config,
        chunks=all_chunks
    )
    
    if success:
        successful_indexes.append(embedding_name)
        print(f'Status: ÉXITO ✅')
    else:
        failed_indexes.append(embedding_name)
        print(f'Status: FALLO ❌')
    
    print('-' * 70)

# Resumen final
print('\nRESUMEN FINAL DE INDEXACIÓN:')
print(f'   • Exitosas: {len(successful_indexes)} - {successful_indexes}')
print(f'   • Fallidas: {len(failed_indexes)} - {failed_indexes}')
print(f'   • Total chunks procesados: {len(all_chunks):,}')

if len(successful_indexes) == len(EMBEDDING_MODELS):
    print('\n🎉 ¡TODAS LAS INDEXACIONES COMPLETADAS EXITOSAMENTE!')
    print('\n💾 PERSISTENCIA AUTOMÁTICA: Los datos se guardan automáticamente en Weaviate')
else:
    print(f'\n⚠️ {len(failed_indexes)} indexaciones fallaron.')

# Información para uso posterior
print('\nWEAVIATE_CLASSES = {')
for embedding_name in successful_indexes:
    class_name = EMBEDDING_MODELS[embedding_name]["class_name"]
    print(f'    "{embedding_name}": "{class_name}",')
print('}')


INICIANDO INDEXACIÓN EN WEAVIATE

Procesando: all-MiniLM-L6-v2

INDEXANDO: all-MiniLM-L6-v2
   • Clase: IdearqAllMiniLM
   • Límite de tokens: 512
   • Batch size: 10
   • Chunks a procesar: 9491
Validando chunks...

VALIDACIÓN COMPLETADA:
   • Chunks válidos: 9481
   • Chunks saltados: 10
   • Tasa de éxito: 99.9%
Probando generación de embeddings...


2025-08-31 00:49:00,377 - INFO - HTTP Request: GET http://localhost:8080/v1/schema/IdearqAllMiniLM "HTTP/1.1 200 OK"
2025-08-31 00:49:00,398 - INFO - HTTP Request: GET http://localhost:8080/v1/schema/IdearqAllMiniLM "HTTP/1.1 200 OK"


   • Test embedding: 384D, norma: 1.000
Iniciando indexación de 9481 chunks...


2025-08-31 00:49:01,092 - INFO - HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
2025-08-31 00:49:01,138 - INFO - HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
2025-08-31 00:49:01,747 - INFO - HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
2025-08-31 00:49:01,755 - INFO - HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
2025-08-31 00:49:02,761 - INFO - HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
2025-08-31 00:49:03,251 - INFO - HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
2025-08-31 00:49:03,258 - INFO - HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
2025-08-31 00:49:04,262 - INFO - HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
2025-08-31 00:49:04,748 - INFO - HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
2025-08-31 00:49:04,755 - INFO - HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 

   Procesados 100 chunks...


...

   Procesados 9400 chunks...


2025-08-31 05:09:07,971 - INFO - HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
2025-08-31 05:09:07,978 - INFO - HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
2025-08-31 05:09:08,983 - INFO - HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
2025-08-31 05:09:14,503 - INFO - HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
2025-08-31 05:09:14,510 - INFO - HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
2025-08-31 05:09:15,515 - INFO - HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
2025-08-31 05:09:18,551 - INFO - HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
2025-08-31 05:09:18,558 - INFO - HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
2025-08-31 05:09:19,563 - INFO - HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
2025-08-31 05:09:25,262 - INFO - HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 

   Procesados 9450 chunks...


2025-08-31 05:10:09,225 - INFO - HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
2025-08-31 05:10:09,231 - INFO - HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
2025-08-31 05:10:10,237 - INFO - HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
2025-08-31 05:10:15,649 - INFO - HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
2025-08-31 05:10:15,656 - INFO - HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
2025-08-31 05:10:16,661 - INFO - HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
2025-08-31 05:10:22,293 - INFO - HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
2025-08-31 05:10:22,299 - INFO - HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
2025-08-31 05:10:23,304 - INFO - HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
2025-08-31 05:10:28,895 - INFO - HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 

INDEXACIÓN EXITOSA: e5-large-instruct
Status: ÉXITO ✅
----------------------------------------------------------------------

RESUMEN FINAL DE INDEXACIÓN:
   • Exitosas: 3 - ['all-MiniLM-L6-v2', 'gte-multilingual-base', 'e5-large-instruct']
   • Fallidas: 0 - []
   • Total chunks procesados: 9,491

🎉 ¡TODAS LAS INDEXACIONES COMPLETADAS EXITOSAMENTE!

💾 PERSISTENCIA AUTOMÁTICA: Los datos se guardan automáticamente en Weaviate

WEAVIATE_CLASSES = {
    "all-MiniLM-L6-v2": "IdearqAllMiniLM",
    "gte-multilingual-base": "IdearqGTE",
    "e5-large-instruct": "IdearqE5",
}


In [16]:
# VERIFICAR INDEXACIÓN (OPCIONAL)
print('\nVERIFICANDO INDEXACIÓN:')
#docker-compose -f docker-compose-weaviate.yml up -d
for embedding_name in successful_indexes:
    class_name = EMBEDDING_MODELS[embedding_name]["class_name"]
    
    try:
        collection = client.collections.get(class_name)
        count = collection.aggregate.over_all(total_count=True)
        print(f'   • {class_name}: {count.total_count} documentos indexados')
    except Exception as e:
        print(f'   • {class_name}: Error verificando - {e}')

print('\nIndexación completa')



VERIFICANDO INDEXACIÓN:
   • IdearqAllMiniLM: 9481 documentos indexados
   • IdearqGTE: 9481 documentos indexados
   • IdearqE5: 9481 documentos indexados

Indexación completa


In [17]:
# Cerrar conexión
client.close()
print('Conexión a Weaviate cerrada')

Conexión a Weaviate cerrada
